# Project Report: Hierarchical Motion Planning Using PRM Methods

**Course:** 24179 – Innovative Concepts for Programming Industrial Robots

**Instructor:** Prof. Dr. Björn Hein

**Institution:** Karlsruhe Institute of Technology (KIT)

**Semester:** Summer Term 2025

**Submission Date:** July 25, 2025

**Authors:**
Simon Boes
and
Mert Baran Ünal


## Technical Retrospective on the Project Progress

As part of our project on the hierarchical PRM method, we received the task description on **June 26, 2025**. The goal was to develop a working hierarchical planning algorithm based on the PRM approach and to evaluate it with respect to different degrees of freedom.

### Week 1 – Theoretical Research and Planning Phase

During the first week, the focus was on a solid **theoretical foundation**. We analyzed the underlying principles of classical PRM as well as the extension to a **hierarchical planning method**, in which a global planner and several local planners (subplanners) work together. Simultaneously, we began drafting the software architecture and strategies for abstraction.

### Week 2 – Initial Implementation Attempts

In the second week, we developed early **implementations of both planning layers**. These initial versions appeared to function correctly — especially the global planner, which generated consistent and valid paths. However, detailed interaction between the global and local planners had not yet been fully tested at this point.

![First visualization of the hierarchical PRM](./images/Fat_Bottleneck_HierLazy.png)

### Week 3 – Error Analysis and External Issues

In week three, we conducted a more comprehensive **testing phase**, during which we discovered that our **local planner was not working as intended**. The resulting paths from the local planner almost always matched those of the global planner — indicating a lack of decoupling and meaningful interaction between the layers. The local planner was effectively bypassed, since the global planner often produced an optimal path on its own.

Simultaneously, we encountered a major setback: pulling the latest version from the GitHub repository rendered our working environment **unusable**, as the external version contained faulty components. Believing the errors to be our own, we **discarded our previously functional code completely** and started over — resulting in a significant and avoidable loss of time.

![Adding a faulty GitHub commit](./images/adding_latest_lecture_setup.png)

### Week 4 – Redevelopment and Final Structure

Following that, we decided on a complete **restart of the PRM implementation**, this time with a highly modular architecture and clear separation of concerns. This solution proved technically robust and was significantly more stable than previous approaches.

![First functional PRM implementation with subplanner](./images/Hierarchical_PRM_Implementation.png)

However, due to the structural redesign, we deviated in some aspects from the originally provided simulation framework, which led to the consequence that **integration into the pre-existing visualization was no longer fully possible**.

![Custom animation of a 2 DoF setup](./images/Simulation_2DoF.png)

### Limitations During Evaluation

One central objective of the project — the **comparative study of performance for 4, 8, and 12 degrees of freedom** using different PRM variants — could therefore **not be fully achieved**.

Although parts of our implementation ran successfully, using the provided simulation framework often resulted in **inconsistent collisions within the visualization**, **extremely long computation times**, or even **complete failure due to timeouts**. These issues prevented a reliable and comparable evaluation of the intended test cases.

## Conclusion

Despite multiple setbacks — both conceptual and technical — we succeeded in developing a fundamentally working hierarchical PRM system. The project clearly highlighted the importance of robust infrastructure, functional reference environments, and modular development practices when tackling complex planning problems. A thorough performance test across varying degrees of freedom would be a logical next step, provided a more stable integration environment is available.


# Final Report – Hierarchical Motion Planning

## 1. Implementation of the Hierarchical Planner

### 1.1 Implementation

#### A. Global Planner: Visibility Roadmap

The global planner is based on a visibility graph. Nodes (guards) are placed in free space, and edges are created only if there is a direct line of sight. This results in a global and efficient structure for coarse path planning.

#### B. Local Planner: BasicPRM and LazyPRM

For each segment $(q_i, q_{i+1})$, a new local PRM instance is constructed. Two variants were tested:

- **BasicPRM**: with immediate collision checking during roadmap construction
- **LazyPRM**: with deferred collision checking upon path usage

The local planner operates within a confined region (bounding box) around the segment:

$$
\text{local\_limits}_x = \left[ \max(c_x - p, x_{\min}),\ \min(c_x + p, x_{\max}) \right]
$$

$$
\text{local\_limits}_y = \left[ \max(c_y - p, y_{\min}),\ \min(c_y + p, y_{\max}) \right]
$$

---

### 1.2 Test Environments

- **2-DoF**: For validating basic functionality
- **4-DoF**: Revealed limitations due to fixed arm lengths
- **Custom Robot Class**: Faster but flawed (collisions were ignored)

Result: Realistic test environments with more than 4 DoF were not reliably usable due to numerical instability in our implementation.

---

### 1.3 Complex Scenarios

Goal: Path planning in 5 complex environments with 4, 8, and 12 DoF

#### 1.3.1 Benchmarking

Planned metrics:

- **Path Length:**
  $$
  L = \sum_{i=0}^{N-1} \| q_{i+1} - q_i \|
  $$
- **Number of Nodes/Edges**
- **Planning Time**
- **Success Rate per Segment**

![Benchmarking of PRM Methods](./images/benchmark_overview.png)

**Discussion of Results**

The benchmark results show clear differences among the tested PRM variants in terms of path length, Euclidean length, runtime, and roadmap size.

### Path Length and Euclidean Distance
The hierarchical methods show distinct behaviors:
- **Hierarchical-Lazy** generates significantly longer paths in multiple scenarios, likely due to LazyPRM’s deferred edge validation, which tends to accept suboptimal connections early and check them later.
- **Hierarchical-Basic**, by contrast, typically produces shorter and more efficient paths. In some cases — e.g., *U-Shape* or *Corridor* — it reaches a quality nearly equal to VisibilityPRM.

### Runtime Behavior
In simple environments, planning times for all methods remain under 0.05 s. However, **Hierarchical-Basic** requires significantly more time in complex scenes such as *Random Obstacles*, with up to **0.07 s**. This may be due to inefficient parameters (e.g., too few nodes).

### Roadmap Size
Roadmap size is highly dependent on the subplanner:
- **LazyPRM** generates very large graphs — often with over 300 nodes — due to repeated edge checking, even when a simpler graph would have sufficed.
- **Hierarchical-Basic** remains significantly more compact and demonstrates controlled behavior with similar path quality.

---

### Suggested Improvement: Iterative Node Growth

A major limitation of all tested planners is the use of a **fixed node count**, regardless of planning success. This results in either:
- unnecessarily large graphs (overhead), or
- planning failure, even though a solution might exist with more nodes.

**We therefore propose an adaptive method** where the number of nodes is increased iteratively until a path is found.

---

#### 1.3.2 Animations

Planned visualizations include:

- Spline interpolation:
  $$
  \gamma(t) = \sum B_i(t) \cdot q_i
  $$
- Animated 2D path visualizations

Robust animations for realistic 6+ DoF models were not feasible.

#### 1.3.3 Discussion of Complex Tasks

Despite technical limitations, the hierarchical planner is theoretically well-suited for:

- Modularizing tasks into subproblems
- Error analysis at the segment level
- Interchangeability of local planners

---

## 2. Theoretical Classification and Path Utilization

### 2.1 Single vs. Multi-Query

- **BasicPRM & LazyPRM**: designed as multi-query methods
- **Within the hierarchical setup**: the local planners behave like single-query instances, since they are re-initialized for each segment

Overall, the hierarchical planner forms a **hybrid architecture**, combining a multi-query global planner with segment-based single-query components.

---

### 2.2 Motion Commands for Robot Programs

A resulting path consists of discrete waypoints $q_i$. These could be executed directly using standard motion commands such as `PTP q[i]` or `LIN q[i]` — for example, in KUKA KRL or URScript.

**However:** The paths were:

- planned without dynamic constraints (v, a)
- generated without trajectory optimization
- possibly containing discontinuities between segments

Thus, **direct execution is technically feasible but not advisable**, as it may lead to jerky motion, collisions, or inefficient trajectories.

---

### 2.3 Dynamically Feasible Execution

To execute a planned path smoothly and meaningfully, the following steps are necessary:

1. **Interpolation** of the discrete waypoints using splines:
   $$
   \gamma(t) = \sum B_i(t) q_i
   $$

2. **Velocity and acceleration limits**:
   $$
   \|\dot{\gamma}(t)\| \leq v_{\max},\quad \|\ddot{\gamma}(t)\| \leq a_{\max}
   $$

3. **Time parameterization** to synchronize all joints

4. **Collision checking along the interpolated path**, not just at discrete points

This results in more physically consistent, smoother, and safer robot motion.

---

### 2.4 Improving Path Quality

To improve the final path, the following methods are recommended:

- **Trajectory smoothing** (e.g., CHOMP, STOMP)
- **Post-processing** to reduce path length
- **Segment-wise resampling** with adaptive density
- **Cost functions** to optimize for energy, time, or safety

These steps significantly enhance both the executability and industrial applicability of the paths.

---

## 3. Conclusion

The developed hierarchical planner shows strong conceptual benefits:

- Separation of global and local planning
- Interchangeable subplanners
- Structured error analysis at the segment level

In practice, however, numerical and kinematic limitations prevented a full evaluation. Nevertheless, the system provides a robust and modular foundation for real robotic systems or improved simulation infrastructures.


# Projektbericht: Hierarchische Bewegungsplanung mit PRM-Verfahren

**Kurs:** 24179 – Innovative Konzepte zur Programmierung von Industrierobotern

**Dozent:** Prof. Dr. Björn Hein

**Institution:** Karlsruher Institut für Technologie (KIT)

**Semester:** Sommersemester 2025

**Abgabedatum:** 25. Juli 2025

**Autoren:**
Simon Boes und
Mert Baran Ünal


## Technischer Rückblick auf den Projektverlauf

Im Rahmen unserer Projektarbeit zum hierarchischen PRM-Verfahren haben wir am **26.06.2025** die Aufgabenstellung erhalten. Ziel war es, ein funktionierendes hierarchisches Planungsverfahren auf Basis des PRM-Ansatzes zu entwickeln und anschließend hinsichtlich unterschiedlicher Freiheitsgrade zu evaluieren.

### Woche 1 – Theoretische Einarbeitung und Planungsphase

In der ersten Projektwoche lag der Fokus auf einer fundierten **theoretischen Einarbeitung**. Wir analysierten die zugrundeliegenden Prinzipien des klassischen PRM sowie die Erweiterung auf ein **hierarchisches Planungsverfahren**, bei dem ein Hauptplaner und mehrere Subplaner gemeinsam agieren. Gleichzeitig begannen wir mit der Grobplanung der Softwarearchitektur und möglichen Abstraktionsstrategien.

### Woche 2 – Erste Implementierungsansätze

In der zweiten Woche entwickelten wir erste **Implementierungen beider Planner-Ebenen**. Die initialen Versionen schienen funktional korrekt zu sein – insbesondere der Hauptplaner erzeugte konsistente und gültige Pfade. Eine detaillierte Interaktion zwischen Haupt- und Subplaner war zu diesem Zeitpunkt jedoch noch nicht vollständig getestet.

![Erste Visuelleisierung des Hierarchical PRM](./images/Fat_Bottleneck_HierLazy.png)

### Woche 3 – Fehleranalyse und externe Probleme

In Woche drei erfolgte eine umfassendere **Testphase**, in der wir feststellten, dass unser **Subplanner nicht wie vorgesehen arbeitete**. Die resultierenden Pfade des Subplanners entsprachen fast immer exakt denen des Hauptplaners – was auf eine mangelnde Entkopplung und Interaktion zwischen den Ebenen hindeutete. Der Subplanner wurde faktisch umgangen, da der globale Planner meist bereits den optimalen Pfad erzeugte.

Parallel dazu erlitten wir einen erheblichen Rückschlag: Durch das Pullen des aktuellen Stands aus dem GitHub-Repository wurde unsere Arbeitsumgebung **unbrauchbar**, da die externe Version fehlerhafte Komponenten enthielt. In dem Glauben, der Fehler läge bei uns, **verwarfen wir unseren funktionierenden Code vollständig** und begannen neu – was zu einem nicht zu unterschätzenden Zeitverlust führte.

![Hinzufügen eines nicht funktionierenden Github-Commits](./images/adding_latest_lecture_setup.png)

### Woche 4 – Neuentwicklung und finale Struktur

Daraufhin entschieden wir uns für einen vollständigen **Neustart der PRM-Implementierung**, diesmal mit einem stark modularisierten Aufbau und klaren Trennung der Verantwortlichkeiten. Diese Lösung funktionierte technisch einwandfrei und war deutlich stabiler als vorherige Ansätze.

![Erste funktionierende PRM-Implementierung mit Subplaner](./images/Hierarchical_PRM_Implementation.png)

Allerdings wichen wir durch den strukturellen Neuansatz in Teilen von der vorgegebenen Simulationsstruktur ab, was zur Folge hatte, dass **die Integration in die ursprünglich vorgesehene Visualisierung nicht mehr vollständig möglich war**.

![Eigene Animation von 2 DoF](./images/Simulation_2DoF.png)

### Einschränkungen bei der Evaluation

Ein zentrales Ziel der Projektaufgabe – die **Vergleichsstudie der Performance bei 4, 8 und 12 Freiheitsgraden** unter Anwendung unterschiedlicher PRM-Varianten – konnte aus diesem Grund **nicht in vollem Umfang umgesetzt werden**.

Unsere Implementierung ließ sich zwar teilweise ausführen, jedoch kam es bei der Ausführung im mitgelieferten Simulationsframework entweder zu **inkonsistenten Kollisionen innerhalb der Visualisierung**, zu **exorbitanten Rechenzeiten** oder sogar zu vollständigem **Abbruch durch Timeouts**. Diese Probleme verhinderten eine belastbare und vergleichbare Auswertung der geplanten Testfälle.

## Fazit

Trotz mehrfacher Rückschläge – sowohl auf konzeptioneller als auch technischer Ebene – gelang es uns, ein grundsätzlich funktionierendes hierarchisches PRM-System zu entwickeln. Die Projektarbeit zeigte deutlich, wie wichtig robuste Infrastruktur, funktionierende Referenzumgebungen und modulare Entwicklung bei komplexeren Planungsproblemen sind. Ein umfassender Test der Performance hinsichtlich verschiedener Freiheitsgrade wäre ein logischer nächster Schritt bei stabilerer Integrationsumgebung.


# Abschlussbericht – Hierarchische Bewegungsplanung

## 1. Umsetzung des hierarchischen Planers

### 1.1 Implementierung

#### A. Hauptplaner: Visibility Roadmap

Der Hauptplaner basiert auf einem Sichtbarkeitsgraph. Knoten (Guards) werden in freiem Raum platziert, Kanten nur bei direkter Sichtverbindung erzeugt. So entsteht eine globale, effiziente Struktur zur Grobplanung.

#### B. Interner Planer: BasicPRM und LazyPRM

Für jedes Segment $(q_i, q_{i+1})$ wird lokal eine neue PRM-Instanz aufgebaut. Zwei Varianten wurden getestet:

- **BasicPRM**: mit sofortiger Kollisionserkennung beim Aufbau
- **LazyPRM**: mit verzögerter Kollisionserkennung bei Pfadnutzung

Der lokale Planer arbeitet in einem begrenzten Raum (Bounding Box) um das Segment:

$$
\text{local\_limits}_x = \left[ \max(c_x - p, x_{\min}),\ \min(c_x + p, x_{\max}) \right]
$$

$$
\text{local\_limits}_y = \left[ \max(c_y - p, y_{\min}),\ \min(c_y + p, y_{\max}) \right]
$$

---

### 1.2 Testumgebungen

- **2-DoF**: Zur Validierung grundlegender Funktion
- **4-DoF**: Zeigte Einschränkungen durch feste Armlängen
- **Eigene Roboterklasse**: Schneller, aber fehlerhaft (Kollisionen wurden ignoriert)

Ergebnis: Realistische Testumgebungen mit mehr als 4 DoF waren aufgrund numerischer Instabilität in unserer Implementierung nicht zuverlässig nutzbar.

---

### 1.3 Komplexe Szenarien

Ziel: Planung in 5 komplexen Umgebungen mit 4, 8 und 12 DoF

#### 1.3.1 Benchmarking

Geplante Metriken:

- **Pfadlänge:**
  $$
  L = \sum_{i=0}^{N-1} \| q_{i+1} - q_i \|
  $$
- **Anzahl Knoten/Kanten**
- **Planungszeit**
- **Erfolgsrate je Segment**

![Benchmarking der PRM-Verfahren](./images/benchmark_overview.png)

**Diskussion der Ergebnisse**

Die Benchmarkauswertung zeigt deutliche Unterschiede zwischen den getesteten PRM-Verfahren hinsichtlich Pfadlänge, euklidischer Länge, Laufzeit und Roadmap-Größe.

### Pfadlänge und euklidische Länge
Die hierarchischen Verfahren zeigen unterschiedliche Verhaltensweisen:
- **Hierarchical-Lazy** erzeugt in mehreren Szenarien signifikant längere Pfade, was auf die verzögerte Kantenvalidierung des LazyPRM zurückzuführen ist. Diese führt dazu, dass suboptimale Verbindungen früh akzeptiert und erst später geprüft werden.
- **Hierarchical-Basic** hingegen produziert meist kürzere und effizientere Pfade. In einigen Szenen, z. B. *U-Shape* oder *Corridor*, erreicht es nahezu die Qualität des VisibilityPRM.

### Laufzeitverhalten
In einfachen Umgebungen bleiben die Planungszeiten aller Verfahren unter 0.05 s. Allerdings zeigt sich, dass **Hierarchical-Basic** in komplexeren Szenen wie *Random Obstacles* mit bis zu **0.07 s** signifikant länger benötigt. Dies kann durch ineffiziente Parametrierung (z. B. zu geringe Knotenzahl) bedingt sein.

### Roadmap-Größe
Die Roadmap-Größe ist stark abhängig vom eingesetzten Subplanner:
- **LazyPRM** erzeugt durch das wiederholte Kantenprüfen sehr große Graphen – oft mit über 300 Knoten – auch wenn ein einfacherer Graph ausgereicht hätte.
- **Hierarchical-Basic** bleibt deutlich kompakter und zeigt kontrolliertes Verhalten bei ähnlicher Pfadqualität.

---

### Verbesserungsvorschlag: Iteratives Knotenwachstum

Ein klarer Schwachpunkt aller getesteten Planer ist die Verwendung einer **festen Knotenzahl**, unabhängig vom Planungserfolg. Das führt entweder zu:
- unnötig großen Graphen (Overhead), oder
- dem Fehlschlagen der Pfadsuche, obwohl mit mehr Knoten ein Pfad möglich wäre.

**Vorgeschlagen wird daher ein adaptives Verfahren**, bei dem die Knotenzahl iterativ erhöht wird, bis ein Pfad gefunden wird:



#### 1.3.2 Animationen

Geplante Darstellung über:

- Spline-Interpolation:
  $$
  \gamma(t) = \sum B_i(t) \cdot q_i
  $$
- Animierte Visualisierung im 2D-Raum

Robuste Animationen für realistische 6+ DoF Modelle waren nicht möglich.

#### 1.3.3 Diskussion komplexer Aufgaben

Trotz technischer Limitierungen ist der hierarchische Planer theoretisch gut geeignet für:

- Modularisierung in Subaufgaben
- Fehlerdiagnose auf Segmentebene
- Austauschbarkeit der Subplaner

---

## 2. Theoretische Einordnung und Pfadverwendung

### 2.1 Single- vs. Multi-Query

- **BasicPRM & LazyPRM**: konzipiert als Multi-Query-Verfahren
- **Im hierarchischen Aufbau** jedoch: lokale Planer arbeiten wie Single-Query-Instanzen, da sie pro Segment neu erzeugt werden

Insgesamt handelt es sich beim hierarchischen Planer um eine **hybride Struktur**, bei der ein Multi-Query-Hauptplaner mit segmentbasierten Single-Query-Komponenten kombiniert wird.

---

### 2.2 Bewegungsbefehle für Roboterprogramme

Ein Ergebnispfad besteht aus diskreten Wegpunkten $q_i$. Diese könnten direkt mit standardisierten Bewegungsbefehlen wie `PTP q[i]` oder `LIN q[i]` ausgeführt werden – z. B. in KUKA KRL oder URScript.

**Aber:** Die Pfade wurden:

- ohne Dynamikgrenzen (v, a) geplant
- ohne Trajektorienoptimierung erzeugt
- mit potenziellen Kantendiskontinuitäten zwischen Segmenten

Daher ist die **direkte Ausführung zwar möglich, aber nicht sinnvoll**, da sie zu ruckhaften Bewegungen, Kollisionen oder ineffizienter Trajektorienführung führen kann.

---

### 2.3 Dynamisch sinnvolle Ausführung

Um einen geplanten Pfad sinnvoll und flüssig abzufahren, sind folgende Schritte nötig:

1. **Interpolation** der diskreten Wegpunkte mittels Splines:
   $$
   \gamma(t) = \sum B_i(t) q_i
   $$

2. **Geschwindigkeits- und Beschleunigungsbegrenzung**:
   $$
   \|\dot{\gamma}(t)\| \leq v_{\max},\quad \|\ddot{\gamma}(t)\| \leq a_{\max}
   $$

3. **Zeitparametrierung** zur Synchronisation aller Gelenke

4. **Kollisionstest entlang des interpolierten Pfads**, nicht nur an den diskreten Punkten

Dies führt zu physikalisch konsistenteren, weicheren und sichereren Bewegungen.

---

### 2.4 Verbesserung der Pfadqualität

Zur Optimierung des Ergebnispfades bieten sich folgende Methoden an:

- **Trajektorienglättung** (z. B. CHOMP, STOMP)
- **Reduktion der Pfadlänge** durch Post-Processing
- **Segmentweises Neusampling** mit adaptiver Dichte
- **Kostenfunktionen** zur Optimierung bzgl. Energie, Zeit oder Sicherheit

Diese Schritte verbessern sowohl die Ausführbarkeit als auch die industrielle Nutzbarkeit erheblich.

---

## 3. Fazit

Der entwickelte hierarchische Planer zeigt konzeptionell starke Ansätze:

- Trennung von globaler und lokaler Planung
- Austauschbare Subplaner
- Strukturierte Fehleranalyse auf Segmentebene

In der Praxis verhinderten jedoch numerische und kinematische Limitationen eine vollständige Evaluation. Für reale Robotersysteme oder verbesserte Simulationsinfrastrukturen bietet das System jedoch eine robuste, modulare Basis.
